In [1]:
import numpy as np
import pandas as pd
from datetime import datetime,timedelta
from datetime import date
import scipy.stats as st

In [2]:
df=pd.read_excel("/home/user/Desktop/env2/VAR/mvp_3/mixed_type.xlsx")

xls = pd.ExcelFile('/home/user/Desktop/env2/VAR/mvp_3/interest_rates.xlsx')
LIBOR_int=pd.read_excel(xls,"LIBOR")
LIBOR_int.index=LIBOR_int["Date"]

SOFR_int=pd.read_excel(xls,"SOFR")
SOFR_int.index=SOFR_int["Date"]

SONIA_int=pd.read_excel(xls,"SONIA")
SONIA_int.index=SONIA_int["Date"]

TIIE_int=pd.read_excel(xls,"TIIE")
TIIE_int.index=TIIE_int["Date"]

BSBY_int=pd.read_excel(xls,"BSBY")
BSBY_int.index=BSBY_int["Date"]
#print(TIIE_int)
df

,Loan ID,Notional Amt,Loan Nature,Start Date,Maturity Date,Comp_Freq,Duration (Period),Base Rate,ARR,Client Spread (LIBOR),Client Spread (ARR),Currency
0,1,1000000,Fixed,2019-12-31,2020-06-30,Monthly,6,LIBOR,SOFR,1.5,1.5,USD
1,2,1000000,Fixed,2019-12-31,2020-12-31,Monthly,12,LIBOR,SOFR,1.5,1.5,USD
2,3,1000000,Fixed,2019-12-31,2022-12-31,Monthly,36,LIBOR,SOFR,1.5,1.5,USD
3,4,1000000,Fixed,2019-12-31,2024-12-31,Monthly,60,LIBOR,SOFR,1.5,1.5,USD
4,5,1000000,Fixed,2019-12-31,2029-12-31,Monthly,120,LIBOR,SOFR,1.5,1.5,USD
5,6,10000000,Floating,2019-04-02,2019-06-30,Monthly,3,LIBOR,SOFR,1.5,1.5,USD
6,7,10000000,Floating,2019-04-02,2019-09-30,Monthly,6,LIBOR,SOFR,1.5,1.5,USD
7,8,10000000,Floating,2019-04-02,2020-03-31,Monthly,12,LIBOR,SOFR,1.5,1.5,USD
8,9,10000000,Floating,2019-04-02,2019-09-30,Quarterly,2,LIBOR,SOFR,1.5,1.5,USD
9,10,10000000,Floating,2019-04-02,2020-03-31,Quarterly,4,LIBOR,SOFR,1.5,1.5,USD


In [3]:
base_rate_calc={"Monthly":"1M","Quarterly":"3M","Six Months":"6M","Annually":"12M"}
time_duration_calc= {"Monthly":1,"Quarterly":3,"Six Months":6,"Annually":12}
effective_col_calc={"Monthly":12,"Quarterly":4,"Six Months":2,"Annually":1}
mapping_dict={"LIBOR":LIBOR_int,"SOFR":SOFR_int,"SONIA":SONIA_int,"TIIE":TIIE_int,"BSBY":BSBY_int}
date_interval={"Monthly":30,"Quarterly":90,"Six Months":180,"Annually":360}

In [4]:
def numOfDays(date1, date2):
    "calculates number of days between two dates"
    return (date2-date1).days

def get_interest_rate(int_df,start_date,tenor):
    "returns interest rate for a particular date"
    interest_rate=int_df[int_df["Date"]==start_date][tenor].values
    return interest_rate

In [5]:
def fixed_rate_calculation(notional,start_date,maturity_date,comp_freq,duration,base_rate,arr,spread_libor,spread_arr,currency,df_cal_fixed,ARR_dict):
    "calculates all interest and impact calculation for fixed rate portfolio"
    
    start_date, maturity_date=pd.to_datetime(start_date), pd.to_datetime(maturity_date)
    tenor= base_rate_calc[comp_freq]
    
    ####### Base Rate interest calculation #########
    int_df=mapping_dict[base_rate]
    interest_rate=get_interest_rate(int_df,start_date,tenor)
    while interest_rate.size==0:
        start_date= start_date+ timedelta(days=-1)
        #print(start_date)
        interest_rate=get_interest_rate(int_df,start_date,tenor)
    df_cal_fixed["Effective LIBOR"].append((1+(interest_rate[0]+spread_libor)/100)**(1/effective_col_calc[comp_freq])-1)
    df_cal_fixed["LIBOR Interest"].append(round(float(df_cal_fixed["Effective LIBOR"][-1])*notional,2))
    df_cal_fixed["LIBOR Total Interest"].append((notional*(1+df_cal_fixed["Effective LIBOR"][-1])**duration)-notional)
    no_of_days= numOfDays(start_date,maturity_date)
    df_cal_fixed["YIELD P.A. % (LIBOR+CLIENT SPREAD)"].append(((((df_cal_fixed["LIBOR Total Interest"][-1]/notional)+1)**(365/no_of_days))-1)*100)
    
    ############ ARR Interest Calculation ##########
    int_df=mapping_dict[arr]
    interest_rate=get_interest_rate(int_df,start_date,tenor)
    while interest_rate.size==0:
        start_date= start_date+ timedelta(days=-1)
        #print(start_date)
        interest_rate=get_interest_rate(int_df,start_date,tenor)
    df_cal_fixed["Effective ARR"].append((1+(interest_rate[0]+spread_arr)/100)**(1/effective_col_calc[comp_freq])-1)
    df_cal_fixed["ARR Interest"].append(round(float(df_cal_fixed["Effective ARR"][-1])*notional,2))
    df_cal_fixed["ARR Total Interest"].append((notional*(1+df_cal_fixed["Effective ARR"][-1])**duration)-notional)
    no_of_days= numOfDays(start_date,maturity_date)
    df_cal_fixed["YIELD P.A. % (ARR+CLIENT SPREAD)"].append(((((df_cal_fixed["ARR Total Interest"][-1]/notional)+1)**(365/no_of_days))-1)*100)
    
    ########## ARR Calculated spread and interest calculation ########
    df_cal_fixed["NET NEUTRALITY YIELD PREMIUM"].append(df_cal_fixed["YIELD P.A. % (ARR+CLIENT SPREAD)"][-1]-df_cal_fixed["YIELD P.A. % (LIBOR+CLIENT SPREAD)"][-1])
    df_cal_fixed["NET NEUTRALITY YIELD PREMIUM (BP)"].append((df_cal_fixed["NET NEUTRALITY YIELD PREMIUM"][-1]*notional)/100)
    df_cal_fixed["Impact (BP)"].append(((df_cal_fixed["ARR Total Interest"][-1]-df_cal_fixed["LIBOR Total Interest"][-1])/notional)*10000)
    df_cal_fixed["ARR Spread Cal"].append(((((df_cal_fixed["Effective LIBOR"][-1]+1)**effective_col_calc[comp_freq])-1)*100)-interest_rate[0])
    df_cal_fixed["Effective ARR Cal"].append((1+(interest_rate[0]+df_cal_fixed['ARR Spread Cal'][-1])/100)**(1/effective_col_calc[comp_freq])-1)
    df_cal_fixed["Interest Calc"].append(round(float(df_cal_fixed["Effective ARR Cal"][-1])*notional,2))
    df_cal_fixed["Total Interest Calc"].append((notional*(1+df_cal_fixed["Effective ARR Cal"][-1])**duration)-notional)
    
    
    ARR_dict[arr].append(df_cal_fixed["ARR Spread Cal"][-1])
    return df_cal_fixed,ARR_dict 

def adjusted_ARR_interest_calculation(df_cal_fixed,ARR_dict,notional,arr,start_date,duration,comp_freq,i):
    
    tenor= base_rate_calc[comp_freq]
    int_df=mapping_dict[arr]
    interest_rate=get_interest_rate(int_df,start_date,tenor)
    while interest_rate.size==0:
        start_date= start_date+ timedelta(days=-1)
        #print(start_date)
        interest_rate=get_interest_rate(int_df,start_date,tenor)
    df_cal_fixed["ARR Spread Adj"].append(round(np.mean(ARR_dict[arr]),4))
    df_cal_fixed["Effective ARR Adj"].append((1+(interest_rate[0]+df_cal_fixed['ARR Spread Adj'][-1])/100)**(1/effective_col_calc[comp_freq])-1)
    df_cal_fixed["Interest Adj"].append(round(float(df_cal_fixed["Effective ARR Adj"][-1])*notional,2))
    df_cal_fixed["Total Interest Adj"].append((notional*(1+df_cal_fixed["Effective ARR Adj"][-1])**duration)-notional)
    df_cal_fixed["Impact"].append(df_cal_fixed["Total Interest Adj"][-1] - df_cal_fixed["Total Interest Calc"][i])
    df_cal_fixed["EMI ARR"].append((notional*df_cal_fixed["Effective LIBOR"][i]*(1+df_cal_fixed["Effective LIBOR"][i])**duration)/(((1+df_cal_fixed["Effective LIBOR"][i])**duration)-1))
    return df_cal_fixed                                   

In [6]:
inter_dict_base={}
inter_dict_ARR={}
inter_dict_ARR_adj={}

def floating_rate_calculation(loan_id,notional,start_date,maturity_date,comp_freq,duration,base_rate,arr,spread_libor,spread_arr,currency,df_cal_floating):
    ######## calculation of interest for floating rate portfolio #########
    
    inter_dict_base[loan_id]=[]
    inter_dict_ARR[loan_id]=[]
    inter_dict_ARR_adj[loan_id]=[]
    interest_monthly=[]
    start_date_orig=start_date
    for j in range(0,duration):
        ####Calculation of interest with Base Rate######

        start_date= start_date_orig+timedelta(days=int(j)*date_interval[comp_freq])
        int_df=mapping_dict[base_rate]
        tenor= base_rate_calc[comp_freq]
        interest_rate=get_interest_rate(int_df,start_date,tenor)
        while interest_rate.size==0:
            start_date= start_date+ timedelta(days=-1)
            #print(start_date)
            interest_rate=get_interest_rate(int_df,start_date,tenor)
            

        effective_rate= ((1+(interest_rate[0]+spread_libor)/100)**(1/effective_col_calc[comp_freq]))-1
        inter_dict_base[loan_id].append(notional*effective_rate)
        
        ####Calculation of interest with ARR Rate######
        int_df=mapping_dict[arr]
        interest_rate=get_interest_rate(int_df,start_date,tenor)

        while interest_rate.size==0:
            start_date= start_date+ timedelta(days=-1)
            #print(start_date)
            interest_rate=get_interest_rate(int_df,start_date,tenor)
            

        effective_rate= ((1+(interest_rate[0]+spread_arr)/100)**(1/effective_col_calc[comp_freq]))-1
        inter_dict_ARR[loan_id].append(notional*effective_rate)
    
    df_cal_floating["LIBOR Total Interest"].append(np.sum(inter_dict_base[loan_id]))
    df_cal_floating["ARR Total Interest"].append(np.sum(inter_dict_ARR[loan_id]))
    no_of_days=numOfDays(start_date,maturity_date)
    df_cal_floating["YIELD P.A. % (LIBOR+CLIENT SPREAD)"].append((((df_cal_floating["LIBOR Total Interest"][-1]/notional)+1)**(365/no_of_days)-1)*100)
    df_cal_floating["YIELD P.A. % (ARR+CLIENT SPREAD)"].append((((df_cal_floating["ARR Total Interest"][-1]/notional)+1)**(365/no_of_days)-1)*100)
    df_cal_floating["NET NEUTRALITY YIELD PREMIUM"].append(df_cal_floating["YIELD P.A. % (ARR+CLIENT SPREAD)"][-1]-df_cal_floating["YIELD P.A. % (LIBOR+CLIENT SPREAD)"][-1])
    df_cal_floating["NET NEUTRALITY YIELD PREMIUM (BP)"].append((df_cal_floating["NET NEUTRALITY YIELD PREMIUM"][-1]*notional)/100)
     
    ###weighted AVG Calculation###

    ARR_Weighted_AVG=(((((df_cal_floating["ARR Total Interest"][-1]/notional)/duration)+1)**effective_col_calc[comp_freq])-1)*100
    df_cal_floating["ARR Spread Adj"].append(((((1+((df_cal_floating["LIBOR Total Interest"][-1]/duration)/notional))**effective_col_calc[comp_freq])-1)*100)-ARR_Weighted_AVG)
    Adjusted_spred=df_cal_floating["ARR Spread Adj"][-1]
    
    ####Calculation of interest with ARR adjusted Rate######
    for j in range(0,duration):
        start_date= start_date_orig+timedelta(days=int(j)*date_interval[comp_freq])
        int_df=mapping_dict[arr]
        interest_rate=get_interest_rate(int_df,start_date,tenor)
        while interest_rate.size==0:
            start_date= start_date+ timedelta(days=-1)
            #print(start_date)
            interest_rate=get_interest_rate(int_df,start_date,tenor)
            

        effective_rate= ((1+(interest_rate[0]+spread_arr+Adjusted_spred)/100)**(1/effective_col_calc[comp_freq]))-1
        inter_dict_ARR_adj[loan_id].append(notional*effective_rate)
    
    df_cal_floating["Total Interest Adj"].append(np.sum(inter_dict_ARR_adj[loan_id]))
    return df_cal_floating

In [7]:
df_cal_fixed={"Effective LIBOR":[],"LIBOR Interest":[],"LIBOR Total Interest":[],"Effective ARR":[],"ARR Interest":[],"ARR Total Interest":[],"YIELD P.A. % (LIBOR+CLIENT SPREAD)":[],"YIELD P.A. % (ARR+CLIENT SPREAD)":[],"NET NEUTRALITY YIELD PREMIUM":[],"NET NEUTRALITY YIELD PREMIUM (BP)":[],"Impact (BP)":[],"ARR Spread Cal":[],"Effective ARR Cal":[],"Interest Calc":[],"Total Interest Calc":[],"ARR Spread Adj":[],"Effective ARR Adj":[],"Interest Adj":[],"Total Interest Adj":[],"Impact":[],"EMI ARR":[]}
df_cal_floating={"LIBOR Total Interest":[],"ARR Total Interest":[],"YIELD P.A. % (LIBOR+CLIENT SPREAD)":[],"YIELD P.A. % (ARR+CLIENT SPREAD)":[],"NET NEUTRALITY YIELD PREMIUM":[],"NET NEUTRALITY YIELD PREMIUM (BP)":[],"ARR Spread Adj":[],"Total Interest Adj":[]}

arr=list(df["ARR"].unique())
ARR_dict={}
for item in arr:
    ARR_dict[item]=[]
for i in range(len(df)):
    loan_id,notional,start_date,maturity_date=df.loc[i,"Loan ID"],df.loc[i,"Notional Amt"],df.loc[i,"Start Date"],df.loc[i,"Maturity Date"]
    comp_freq,duration,nature=df.loc[i,"Comp_Freq"],df.loc[i,"Duration (Period)"],df.loc[i,"Loan Nature"]
    base_rate,arr,spread_libor,spread_arr,currency=df.loc[i,"Base Rate"],df.loc[i,"ARR"],df.loc[i,"Client Spread (LIBOR)"],df.loc[i,"Client Spread (ARR)"],df.loc[i,"Currency"]
    if nature=="Fixed":
        df_cal_fixed,ARR_dict=fixed_rate_calculation(notional,start_date,maturity_date,comp_freq,duration,base_rate,arr,spread_libor,spread_arr,currency,df_cal_fixed,ARR_dict)
    if nature=="Floating":
        df_cal_floating = floating_rate_calculation(loan_id,notional,start_date,maturity_date,comp_freq,duration,base_rate,arr,spread_libor,spread_arr,currency,df_cal_floating)
for i in range(len(df)):
    notional,start_date,arr,duration,comp_freq,nature= df.loc[i,"Notional Amt"],df.loc[i,"Start Date"],df.loc[i,"ARR"],df.loc[i,"Duration (Period)"],df.loc[i,"Comp_Freq"],df.loc[i,"Loan Nature"]
    if nature=="Fixed":
        df_cal_fixed=adjusted_ARR_interest_calculation(df_cal_fixed,ARR_dict,notional,arr,start_date,duration,comp_freq,i)
int_df_fixed=pd.DataFrame(df_cal_fixed)
int_df_floating= pd.DataFrame(df_cal_floating)
int_df_fixed

,Effective LIBOR,LIBOR Interest,LIBOR Total Interest,Effective ARR,ARR Interest,ARR Total Interest,YIELD P.A. % (LIBOR+CLIENT SPREAD),YIELD P.A. % (ARR+CLIENT SPREAD),NET NEUTRALITY YIELD PREMIUM,NET NEUTRALITY YIELD PREMIUM (BP),...,ARR Spread Cal,Effective ARR Cal,Interest Calc,Total Interest Calc,ARR Spread Adj,Effective ARR Adj,Interest Adj,Total Interest Adj,Impact,EMI ARR
0,0.002679,2678.92,16181.578262,0.002505,2504.6,15122.012371,3.271608,3.055768,-0.215840,-2158.397087,...,1.71523,0.002679,2678.92,16181.578262,1.7152,0.002679,2678.9,16181.430651,-0.147611,168232.856244
1,0.002679,2678.92,32625.000000,0.002505,2504.6,30472.700000,3.253443,3.038819,-0.214624,-2146.237247,...,1.71523,0.002679,2678.92,32625.000000,1.7152,0.002679,2678.9,32624.700000,-0.300000,84791.534263
2,0.002679,2678.92,101102.897619,0.002505,2504.6,94232.152842,3.259475,3.044448,-0.215028,-2150.275282,...,1.71523,0.002679,2678.92,101102.897619,1.7152,0.002679,2678.9,101101.937936,-0.959683,29175.928210
3,0.002679,2678.92,174121.865290,0.002505,2504.6,161936.657140,3.258871,3.043884,-0.214987,-2149.870795,...,1.71523,0.002679,2678.92,174121.865290,1.7152,0.002679,2678.9,174120.159752,-1.705539,18064.262843
4,0.002679,2678.92,378562.154552,0.002505,2504.6,350096.795206,3.259777,3.044730,-0.215048,-2150.477582,...,1.71523,0.002679,2678.92,378562.154552,1.7152,0.002679,2678.9,378558.149535,-4.005017,9755.500059
